In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")  # Keep the original neo4j+s:// format
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

print(f"Attempting connection to: {NEO4J_URI}")
print(f"Username: {NEO4J_USERNAME}")

from langchain_community.graphs import Neo4jGraph

graph= Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

Attempting connection to: neo4j+ssc://b74b1fa9.databases.neo4j.io
Username: neo4j


In [13]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [14]:
from langchain.chat_models import init_chat_model
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain


chat_model = init_chat_model("groq:openai/gpt-oss-20b")


chain = GraphCypherQAChain.from_llm(graph=graph, llm=chat_model, verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000020F1AC7A650>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020F1AC6DA10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020F1AC25610>, model_name=

In [15]:
response = chain.invoke({"query":"How many artist are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(p) AS numArtists;
Full Context:
[{'numArtists': 1239}]

> Finished chain.


{'query': 'How many artist are there?', 'result': '1239 artists.'}

In [16]:
examples = [
    {
        "question": "How many artist are there?",
        "query": "MATCH (p:Person) RETURN count(p) AS artistCount;"
    },
    {
        "question": "Who were the actors of the movie Casino?",
        "query": "MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Casino'}) RETURN p.name;"
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m) AS movieCount;"
    },
    {
        "question": "List all the genres of the movie Schindler's List.",
        "query": "MATCH (m:Movie {title: 'Schindler\\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name;"
    },
    {
        "question": "Which actors have acted in both comedy and action movies?",
        "query": "MATCH (p:Person)-[:ACTED_IN]->(m1:Movie)-[:IN_GENRE]->(:Genre {name: 'Comedy'}), (p)-[:ACTED_IN]->(m2:Movie)-[:IN_GENRE]->(:Genre {name: 'Action'}) RETURN DISTINCT p.name;"
    }
]

In [17]:
response = chain.invoke({"query":"How many artist are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(p) AS artistCount;
Full Context:
[{'artistCount': 1239}]

> Finished chain.


{'query': 'How many artist are there?', 'result': 'There are 1239 artists.'}

In [18]:
response = chain.invoke({"query":"List all the genres of the movie Balto."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Balto'})-[:IN_GENRE]->(g:Genre)
RETURN g.name;
Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Animation'}, {'g.name': 'Children'}]

> Finished chain.


{'query': 'List all the genres of the movie Balto.',
 'result': 'Adventure, Animation, Children'}

In [34]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\n Cypher Query: {query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher Query:",
    input_variables=["question","schema"],
)

In [35]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artist are there?', 'query': 'MATCH (p:Person) RETURN count(p) AS artistCount;'}, {'question': 'Who were the actors of the movie Casino?', 'query': "MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Casino'}) RETURN p.name;"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m) AS movieCount;"}, {'question': "List all the genres of the movie Schindler's List.", 'query': "MATCH (m:Movie {title: 'Schindler\\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name;"}, {'question': 'Which actors have acted in both comedy and action movies?', 'query': "MATCH (p:Person)-[:ACTED_IN]->(m1:Movie)-[:IN_GENRE]->(:Genre {name: 'Comedy'}), (p)-[:ACTED_IN]->(m2:Movie)-[:IN_GENRE]->(:Genre {name: 'Action'}) RETURN DISTINCT p.name;"}], example_prompt=PromptTemplate(input_variables=['query', 'question'],

In [ ]:
print(prompt.format(question="How many artist are there?"))

KeyError: 'title'